In [ ]:
import processscheduler as ps

### TaskDontOverlap constraint
Make two tasks don't overlap, one is scheduled after the other, or the opposite.

In [ ]:
# problem
pb1 = ps.SchedulingProblem("DontOverlapExample", horizon=6)
# tasks
t1 = ps.FixedLengthTask('t1', length=2)
t2 = ps.FixedLengthTask('t2', length=2)
t3 = ps.FixedLengthTask('t3', length=2)
pb1.add_tasks([t1, t2,t3])
# constraints
c1 = ps.TaskStartAt(t2, 1)
c2 = ps.TaskDontOverlap(t2, t3) # t2 and t3 must not overlap
pb1.add_constraints([c1, c2])
# solve
solver1 = ps.SchedulingSolver(pb1, verbosity=False)
solver1.solve()
pb1.render_gantt_matplotlib()